In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_APIKEY'
os.environ['PINECONE_API_KEY'] = 'YOUR_PINECONE_APIKEY'

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

In [ ]:
openAIEmbeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
file_path = "./curry_menu_japanese.csv"

In [ ]:
loader = CSVLoader(file_path=file_path)

In [ ]:
doc_data = loader.load()

In [ ]:
pc = Pinecone()

In [ ]:
index_name = "curryproductinfo"

In [ ]:
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        ) 
    )

In [ ]:
from langchain_pinecone.embeddings import PineconeEmbeddings

In [ ]:
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
    
if index_name in existing_indexes:
    vectorstore = PineconeVectorStore.from_existing_index(index_name, openAIEmbeddings)

In [ ]:
vectorstore.add_documents(doc_data)

In [ ]:
query = "ベジタリアン向けのカレーを教えてください"

In [ ]:
def perform_search(vectorstore, query):
    results = vectorstore.similarity_search_with_relevance_scores(query, k=5)
    for doc, score in results:
        print(f"* [SCORE={score}] [{doc.page_content}]")

In [ ]:
perform_search(vectorstore, query)